In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import torch
from torch import nn

from hparams.hparams_W4G import create_hparams
from model import Model
from datasets import LJDataset, TextMelCollate
from utils import sizeof_fmt, Logger

In [2]:
!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Thu Mar 18 22:04:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:2E:00.0 Off |                  N/A |
| 50%   65C    P0    97W / 250W |      0MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:2F:00.0 Off |                  N/A |
| 30%   

In [3]:
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model

device='cuda'
vocoder_tag = "ljspeech_parallel_wavegan.v3"

# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to(device).eval()
vocoder.remove_weight_norm()
print('done')

from parallel_wavegan.utils import read_hdf5

stats_filename = "/home/rubato/.cache/parallel_wavegan/ljspeech_parallel_wavegan.v3/stats.h5"
mean = read_hdf5(stats_filename, 'mean')
scale = read_hdf5(stats_filename, 'scale')
print('done')

done
done


In [6]:
save_dir = '/data/save/model_W4G'
logger = Logger(save_dir=save_dir, new=False)
print('done')

done


In [8]:
!ls $save_dir

data.json    save_101436  save_102141  save_400000
save_100000  save_101938  save_102658


In [9]:
stt_hparams, tts_hparams = create_hparams()
model = Model(stt_hparams, tts_hparams, mode='inference')
model = model.cuda()
step = 400000

size = sizeof_fmt(4 * sum(p.numel() for p in model.parameters()))
print(f"Model size {size}")

size = sizeof_fmt(4 * sum(p.numel() for p in model.tts.parameters()))
print(f"TTS size {size}")

size = sizeof_fmt(4 * sum(p.numel() for p in model.tts.mel_decoder.parameters()))
print(f"MelDecoder size {size}")

if True:
    model, _, _ = logger.load(step, model, None)
print(step)

print('done')

Model size 13.5MiB
TTS size 13.5MiB
MelDecoder size 11.6MiB
loaded : 400000
400000
done


In [11]:
testset = LJDataset(tts_hparams, split='valid')
collate_fn = TextMelCollate(tts_hparams)

test_loader = torch.utils.data.DataLoader(testset, num_workers=1, 
                          shuffle=False, sampler=None, batch_size=1, pin_memory=False,
                          drop_last=True, collate_fn=collate_fn)
print(test_loader)

In [12]:
def to_cuda(batch):
    batch['text'] = batch['text'].cuda()
    batch['text_lengths'] = batch['text_lengths'].cuda()
    batch['mels'] = batch['mels'].cuda()
    batch['mel_lengths'] = batch['mel_lengths'].cuda()
    
    return batch

In [13]:
!mkdir inference

In [14]:
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import time 
import soundfile as sf

temperature = 1.0
clip = 2
inference_dir = 'inference/W4G/'
!mkdir $inference_dir

model.eval()
vocoder.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        print(i, batch['strings'][0])
        batch = to_cuda(batch)
        samples, time_dict = model.inference(batch['text'][0:1], None, None, temperature=temperature, clip=clip)
        _s = samples.data.cpu().numpy()
        _mel = _s[0].T
        _mel = (_mel - mean) / scale
        mel = torch.from_numpy(_mel).cuda()
        wav = vocoder.inference(mel)
        _wav = wav[:, 0].data.cpu().numpy()
        save_path = inference_dir + 'inference_' + str(i) + '.wav'
        sf.write(save_path, _wav, 22050, subtype='PCM_16')
        
print('done')

0 The Chronicles of Newgate, Volume two. By Arthur Griffiths. Section five: Newgate down to eighteen eighteen, part two.
1 The squalor and uncleanness of the debtors' side was intensified by constant overcrowding.
2 Prisoners were committed to it quite without reference to its capacity.
3 No remonstrance was attended to,
4 no steps taken to reduce the number of committals, and the governor was obliged to utilize the chapel as a day and night room.
5 Besides this, although the families of debtors were no longer permitted to live with them inside the jail,
6 hundreds of women and children came in every morning to spend the day there, and there was no limitation whatever to the numbers of visitors admitted to the debtors' side.
7 Friends arrived about nine a.m., and went out at nine p.m., when as many as two hundred visitors have been observed leaving the debtors' yards at one time.
8 The day passed in revelry and drunkenness. Although spirituous liquors were forbidden,
9 wine and beer mi

80 Persons convicted of publishing libels were still immured in the same rooms with transports and felons.
81 The middle yard, as far as its limits would permit, was appropriated to felons and transports. The wards here were generally very crowded.
82 Each ward was calculated to hold twenty-four, allowing each individual one foot and a half;
83 Quote, a common-sized man, says the keeper, Mr. Newman, can turn in nineteen inches, end quote.
84 These twenty-four could just sleep on the barrack bed; when the number was higher, and it often rose to forty, the surplus had to sleep on the floor.
85 The crowding was in consequence of the delay in removing transports.
86 These often remained in Newgate for six months, sometimes a year, in some cases longer;
87 in one, for seven years -- that of a man sentenced to death, for whom great interest had been made, but whom it was not thought right to pardon.
88 Occasionally the transports made themselves so useful in the jail that they were passed ov

159 and no lodger was rejected, whatever his status, who offered himself and could bring grist to the mill.
160 The luxury of the state side was for a long time open to all who could pay
161 the convicted felon, the transport awaiting removal, the lunatic whose case was still undecided,
162 the misdemeanant tried or untried, the debtor who wished to avoid the discomfort of the crowded debtors' side, the outspoken newspaper editor,
163 or the daring reporter of parliamentary debates.
164 The better class of inmate complained bitterly of this enforced companionship with the vile,
165 association at one time forbidden by custom, but which greed and rapacity long made the rule.
166 The fee for admission to the state side, as fixed by the table of fees, was three guineas, but Mr. Newman declared that he never took more than two.
167 Ten and sixpence a week more was charged as rent for a single bed; where two or more slept in a bed the rent was seven shillings a week each.
168 Prisoners who 

237 and they were exacted to relieve a rich corporation from paying for the maintenance of their own prison.
238 This imposition of fees left prisoners destitute on their discharge, without funds to support them in their first struggle to recommence life,
239 with ruined character, bad habits, and often bad health contracted in the jail.
240 A further and a more iniquitous method of extorting money
241 was still practiced, that of loading newly-arrived prisoners until they paid certain fees.
242 Ironing was still the rule, not only for the convicted, but for those charged with felonies; only the misdemeanants escaped.
243 At the commencement of every sessions, such of the untried as had purchased "easement" of irons were called up and re-fettered,
244 preparatory to their appearance in the Old Bailey. Irons were seldom removed from the convicted until discharge;
245 sometimes the wearer was declared medically unfit, or he obtained release by long good conduct,
246 or the faithful disch

314 It should be peremptorily forbidden to the keeper or any officer to make a pecuniary profit out of the supplies of food, fuel, or other necessaries.
315 No prisoner should be allowed to obtain superior accommodation on the payment of any fees. Fees indeed should be generally abolished, garnish also.
316 No prisoners should in future be ironed, except in cases of misconduct,
317 provided only that their security was not jeopardized, and dependent upon the enforcement of another new rule,
318 which recommended restrictions upon the number of visitors admitted.
319 No wine or beer should be in future admitted into or sold in the jail,
320 except for the use of the debtors, or as medical comforts for the infirmary.
321 Drunkenness, if it ever occurred, should be visited with severe punishment;
322 gaming of all sorts should be peremptorily forbidden under heavy pains and penalties.
323 The feelings of the condemned prisoners should no longer be outraged by their exposure in the chapel,